In [1]:
# Imported packages
import os, glob

import numpy as np
import math
from matplotlib import pyplot as plt
from astroquery import sha
from astropy import coordinates as coord
#from astropy.coordinates import SkyCoord
from astropy import units as u

from urllib.parse import urlencode
from urllib.request import urlretrieve
from IPython.display import Image

/home/murphyj/anaconda3/lib/python3.6/site-packages/astroquery/sha/__init__.py:14: UserWarning: Experimental: SHA has not yet been refactored to have its API match the rest of astroquery.
  warnings.warn("Experimental: SHA has not yet been refactored to have its "


In [2]:
# Loads csv file
line = np.genfromtxt(open('/home/murphyj/Desktop/Coding/SNR_list.csv', "r"), names=True, delimiter=',', dtype=None)

/home/murphyj/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: VisibleDeprecationWarning: Reading unicode strings without specifying the encoding argument is deprecated. Set the encoding, use None for the system default.
  


In [3]:
# Defines Columns as lists
MCSNR = []#0
RA = []#1
DE = []#2
Rad = []#8
kT = []#12
VShock = []#16
Age = []#18
LX = []#4
LIR = []#7

num = line.shape[0]
for n in range(num):
    #print(l)
    name = line[n][0]
    MCSNR.append(name)
MCSNR = np.asarray(MCSNR)

num = line.shape[0]
for n in range(num):
    #print(l)
    name = line[n][1]
    RA.append(name)
RA = np.asarray(RA)

num = line.shape[0]
for n in range(num):
    #print(l)
    name = line[n][2]
    DE.append(name)
DE = np.asarray(DE)

num = line.shape[0]
for n in range(num):
    #print(l)
    name = line[n][8]
    Rad.append(name)
Rad = np.asarray(Rad)

num = line.shape[0]
for n in range(num):
    #print(l)
    name = line[n][12]
    kT.append(name)
kT = np.asarray(kT)

num = line.shape[0]
for n in range(num):
    #print(l)
    name = line[n][17]
    VShock.append(name)
VShock = np.asarray(VShock)

num = line.shape[0]
for n in range(num):
    #print(l)
    name = line[n][20]
    Age.append(name)
Age = np.asarray(Age)

num = line.shape[0]
for n in range(num):
    #print(l)
    name = line[n][4]
    LX.append(name)
LX = np.asarray(LX)

num = line.shape[0]
for n in range(num):
    #print(l)
    name = line[n][7]
    LIR.append(name)
LIR = np.asarray(LIR)

In [4]:
# Decodes bytes to strings
x = [Ra.decode('UTF-8') for Ra in RA]
y = [Dec.decode('UTF-8') for Dec in DE]

# Creates Ra and Dec formats from data
im_ra = []
for n in range(num):
    things = x[n].split()
    Ra = things[0]+'h'+things[1]+'m'+things[2]+'s'
    im_ra.append(Ra)
print(im_ra[0])

im_dec = []
for n in range(num):
    things = y[n].split()
    Dec = things[0]+'d'+things[1]+'m'+things[2]+'s'
    im_dec.append(Dec)
#print(Dec[1])

04h49m20s


In [5]:
# Defines image sizes
im_size_as = Rad * 2
im_size_deg = im_size_as / 3600
im_size_am = im_size_as / 60

In [6]:
# Todo
# Astroquery
im_size = im_size_am*u.arcmin
im_pixels = 1024

In [7]:
test_src_coord = [im_ra[0], im_dec[0]]
print(im_ra[0])

04h49m20s


In [8]:
my_query = sha.query(coord=coord.SkyCoord(im_ra[0], im_dec[0], frame='icrs'), size=0.001)

In [9]:
print(my_query.columns)

<TableColumns names=('reqkey','modedisplayname','wavelength','cra','cdec','filetype','externalname','ptcomment','ra','dec','epoch','equinox','begintime','endtime','minwavelength','maxwavelength','pbcdid','filesize','campid','ra1','dec1','ra2','dec2','ra3','dec3','ra4','dec4','naxis1','naxis2','cdelt1','cdelt2','crota2','crpix1','crpix2','crval1','crval2','ctype1','ctype2','heritagefilename','hasAccess','accessUrl','accessWithAnc1Url')>


In [10]:
my_query['modedisplayname', 'wavelength', 'filetype', 'filesize', 'ptcomment'].pprint(max_lines=100, max_width=-1) 

modedisplayname   wavelength  filetype  filesize                        ptcomment                        
---------------- ----------- --------- --------- --------------------------------------------------------
 MIPS Scan        MIPS 24um   Image     22161600  Mosaic image                                           
 IRAC Map PC      IRAC 3.6um  Image      9622080  Mosaic image (long frametime if data taken in HDR mode)
 IRAC Map PC      IRAC 3.6um  Image      9613440  Mosaic image (long frametime if data taken in HDR mode)
 IRAC Map PC      IRAC 3.6um  Image      9627840  Mosaic image (long frametime if data taken in HDR mode)
 IRAC Map PC      IRAC 4.5um  Image      9613440  Mosaic image (long frametime if data taken in HDR mode)
 IRAC Map PC      IRAC 3.6um  Image      9613440  Mosaic image (long frametime if data taken in HDR mode)
 IRAC Map PC      IRAC 3.6um  Image      9622080  Mosaic image (long frametime if data taken in HDR mode)
 IRAC Map PC      IRAC 4.5um  Image      96278

In [11]:
irac_36_mask = (my_query['wavelength'] == ' IRAC 3.6um')
irac_36_tbl = my_query[irac_36_mask]
irac_36_tbl['modedisplayname', 'wavelength', 'filetype', 'filesize', 'ptcomment'].pprint(max_lines=100, max_width=-1) 

modedisplayname   wavelength  filetype  filesize                        ptcomment                        
---------------- ----------- --------- --------- --------------------------------------------------------
 IRAC Map PC      IRAC 3.6um  Image      9622080  Mosaic image (long frametime if data taken in HDR mode)
 IRAC Map PC      IRAC 3.6um  Image      9613440  Mosaic image (long frametime if data taken in HDR mode)
 IRAC Map PC      IRAC 3.6um  Image      9627840  Mosaic image (long frametime if data taken in HDR mode)
 IRAC Map PC      IRAC 3.6um  Image      9613440  Mosaic image (long frametime if data taken in HDR mode)
 IRAC Map PC      IRAC 3.6um  Image      9622080  Mosaic image (long frametime if data taken in HDR mode)
 IRAC Map PC      IRAC 3.6um  Image      9622080  Mosaic image (long frametime if data taken in HDR mode)
 IRAC Map PC      IRAC 3.6um  Image      9613440  Mosaic image (long frametime if data taken in HDR mode)
 IRAC Map PC      IRAC 3.6um  Image      96220

In [12]:
for u in irac_36_tbl['accessUrl']:
    print("Getting: {}".format(u))
    url = u.strip()
    sha.save_file(url, out_dir='test_files_3.6um/')

Getting:  http://sha.ipac.caltech.edu/applications/Spitzer/SHA/servlet/ProductDownload?DATASET=level2&ID=11734587   
Getting:  http://sha.ipac.caltech.edu/applications/Spitzer/SHA/servlet/ProductDownload?DATASET=level2&ID=11890877   
Getting:  http://sha.ipac.caltech.edu/applications/Spitzer/SHA/servlet/ProductDownload?DATASET=level2&ID=11921087   
Getting:  http://sha.ipac.caltech.edu/applications/Spitzer/SHA/servlet/ProductDownload?DATASET=level2&ID=11642554   
Getting:  http://sha.ipac.caltech.edu/applications/Spitzer/SHA/servlet/ProductDownload?DATASET=level2&ID=11731875   
Getting:  http://sha.ipac.caltech.edu/applications/Spitzer/SHA/servlet/ProductDownload?DATASET=level2&ID=11619503   
Getting:  http://sha.ipac.caltech.edu/applications/Spitzer/SHA/servlet/ProductDownload?DATASET=level2&ID=11661742   
Getting:  http://sha.ipac.caltech.edu/applications/Spitzer/SHA/servlet/ProductDownload?DATASET=level2&ID=11804182   
Getting:  http://sha.ipac.caltech.edu/applications/Spitzer/SHA/s